**This is the task for the second home work (HW2)**

Need to get Armenian MCV dataset and train Armenian ASR model

Quality metric is WER on Armenian MCV test subset.

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=5d63fe10d22e86f6689e2bc721c02ed83d2c65a301f1b68df3ce2ccb41172b68
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3
  libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox-fmt-mp3 libsox3 libwavpack1 sox
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 72

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libsox-fmt-mp3 is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-6nwsqv2z/nemo-toolkit_54078016bfcd48d08c163db8b307a45e
  Running command git clone --filter=blob:none --quiet https://github.com/

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [3]:
import os
import glob
import subprocess
import tarfile
import wget
import copy
from omegaconf import OmegaConf, open_dict


In [4]:
data_dir = 'datasets/'

if not os.path.exists(data_dir):
  os.makedirs(data_dir, exist_ok=True)

if not os.path.exists("scripts"):
  os.makedirs("scripts")

import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager

**Download dataset**

We will use the NeMo script in the scripts directory to download and prepare the Mozilla Common Voice (MCV) dataset for Armenian.

The data preparation script will download the audio files and respective transcripts and then process the audio into mono-channel 16 kHz wave files that can be easily used for training ASR models.

**Hugging Face**

Now, let's download the Mozilla CommonVoice Spanish dataset. We will ignore the larger train file and get just the test part for the purposes of this tutorial. For good results, you will need to get the train files and likely other datasets too, bringing the total to over 1k hours.

Website steps:

Visit https://huggingface.co/settings/profile

Visit "Access Tokens" on list of items.

Create new token - provide a name for the token and "read" access is sufficient.

PRESERVE THAT TOKEN API KEY. You can copy that key for next step.

Visit the HuggingFace Dataset page for [Mozilla Common Voice 16.1](https://huggingface.co/datasets/mozilla-foundation/common_voice_16_1)

There should be a section that asks you for your approval.

Make sure you are logged in and then read that agreement.

If and only if you agree to the text, then accept the terms.

Code steps:

* Now below, run login()

* Paste your preserved HF TOKEN API KEY to the text box."

In [5]:
from huggingface_hub import login
login()

In [6]:
VERSION = "mozilla-foundation/common_voice_16_1"
LANGUAGE = "hy-AM"

In [7]:
tokenizer_dir = os.path.join('tokenizers', LANGUAGE)
manifest_dir = os.path.join('datasets', LANGUAGE, VERSION, LANGUAGE)

In [8]:
# If something goes wrong during data processing, un-comment the following line to delete the cached dataset
# !rm -rf datasets/$LANGUAGE
!mkdir -p datasets

The following cell will download the Japanese MCV corpus, preprocess the audio and prepare manifest files that can be directly used by NeMo models.

We will use the convert_hf_dataset_to_nemo.py script located in the nemo/scripts/speech_recognition dir if you cloned NeMo repo

In [9]:
if not os.path.exists("convert_hf_dataset_to_nemo.py"):
    !wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/speech_recognition/convert_hf_dataset_to_nemo.py

--2024-03-11 13:21:59--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/speech_recognition/convert_hf_dataset_to_nemo.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14735 (14K) [text/plain]
Saving to: ‘convert_hf_dataset_to_nemo.py’

convert_hf_dataset_ 100%[===================>]  14.39K  --.-KB/s    in 0.001s  

2024-03-11 13:21:59 (25.8 MB/s) - ‘convert_hf_dataset_to_nemo.py’ saved [14735/14735]



In [10]:
!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="train" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="validation" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="test" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="other" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="invalidated" \
    ensure_ascii=False \
    use_auth_token=True

/content/convert_hf_dataset_to_nemo.py:346: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_name='hfds_config', config_path=None)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load 

In [11]:
train_manifest = f"{manifest_dir}/train/train_mozilla-foundation_common_voice_16_1_manifest.json"
dev_manifest = f"{manifest_dir}/validation/validation_mozilla-foundation_common_voice_16_1_manifest.json"
test_manifest = f"{manifest_dir}/test/test_mozilla-foundation_common_voice_16_1_manifest.json"
other_manifest = f"{manifest_dir}/other/other_mozilla-foundation_common_voice_16_1_manifest.json"
invalidated_manifest = f"{manifest_dir}/invalidated/invalidated_mozilla-foundation_common_voice_16_1_manifest.json"

In [12]:
train_manifest_full = f"{manifest_dir}/train_full_mozilla-foundation_common_voice_16_1_manifest.json"
!cat $train_manifest $other_manifest $invalidated_manifest > $train_manifest_full

**Hint**: Convert texts to lowercase and remove punctuation to improve WER.

In [13]:
if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tokenizers/process_asr_text_tokenizer.py

--2024-03-11 13:52:05--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16631 (16K) [text/plain]
Saving to: ‘scripts/process_asr_text_tokenizer.py’

process_asr_text_to 100%[===================>]  16.24K  --.-KB/s    in 0.001s  

2024-03-11 13:52:05 (30.4 MB/s) - ‘scripts/process_asr_text_tokenizer.py’ saved [16631/16631]



**Hint**: Play with `VOCAB_SIZE` to improve WER.

In [16]:
TOKENIZER_TYPE = "bpe" # "bpe", "unigram"
VOCAB_SIZE = 128 + 2

In [17]:
!python scripts/process_asr_text_tokenizer.py \
  --manifest=$train_manifest_full,$dev_manifest \
  --vocab_size=$VOCAB_SIZE \
  --data_root=$tokenizer_dir \
  --tokenizer="spe" \
  --spe_type=$TOKENIZER_TYPE \
  --spe_character_coverage=1.0 \
  --no_lower_case \
  --log

INFO:root:Finished extracting manifest : datasets/hy-AM/mozilla-foundation/common_voice_16_1/hy-AM/train_full_mozilla-foundation_common_voice_16_1_manifest.json
INFO:root:Finished extracting manifest : datasets/hy-AM/mozilla-foundation/common_voice_16_1/hy-AM/validation/validation_mozilla-foundation_common_voice_16_1_manifest.json
INFO:root:Finished extracting all manifests ! Number of sentences : 12707
[NeMo I 2024-03-11 13:59:16 sentencepiece_tokenizer:317] Processing tokenizers/hy-AM/text_corpus/document.txt and store at tokenizers/hy-AM/tokenizer_spe_bpe_v130
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokenizers/hy-AM/text_corpus/document.txt --model_prefix=tokenizers/hy-AM/tokenizer_spe_bpe_v130/tokenizer --vocab_size=130 --shuffle_input_sentence=true --hard_vocab_limit=false --model_type=bpe --character_coverage=1.0 --bos_id=-1 --eos_id=-1
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokenizers/hy-AM/text_corpus/docu

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def preprocess_text(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    text = text.lower()

    armenian_punctuation = "։՛՝,․՞…:"
    text = ''.join(char for char in text if char not in armenian_punctuation)
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(text)

# Example usage
input_file_path = '/content/tokenizers/hy-AM/text_corpus/document.txt'
preprocess_text(input_file_path, input_file_path)


**Hint**: Try different models.

In [36]:
model = nemo_asr.models.ASRModel.from_pretrained("nvidia/stt_en_fastconformer_transducer_large", map_location='cuda')

[NeMo W 2024-03-11 14:12:25 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
    The secret `HF_TOKEN` does not exist in your Colab secrets.
    To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
    You will be able to reuse this secret in all of your notebooks.
    Please note that authentication is recommended but still optional to access public models or datasets.
      warnings.warn(
    


(…)t_en_fastconformer_transducer_large.nemo:   0%|          | 0.00/457M [00:00<?, ?B/s]

[NeMo I 2024-03-11 14:12:33 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-03-11 14:12:34 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-03-11 14:12:34 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2024-03-11 14:12:34 features:289] PADDING: 0


[NeMo W 2024-03-11 14:12:35 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2024-03-11 14:12:35 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-03-11 14:12:35 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-03-11 14:12:35 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-03-11 14:12:36 save_restore_connector:263] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_fastconformer_transducer_large/snapshots/63d5e0b706c82b5cc14ba0d9955393af7e57b863/stt_en_fastconformer_transducer_large.nemo.


In [37]:
import torch
import torch.nn as nn

freeze_encoder = True # set to False if dare lol

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

if freeze_encoder:
  model.encoder.freeze()
  model.encoder.apply(enable_bn_se)
  logging.info("Model encoder has been frozen")
else:
  model.encoder.unfreeze()
  logging.info("Model encoder has been un-frozen")

[NeMo I 2024-03-11 14:12:36 <ipython-input-37-180610cd99a3>:20] Model encoder has been frozen


In [38]:
TOKENIZER_DIR = os.path.join(tokenizer_dir, f"tokenizer_spe_{TOKENIZER_TYPE}_v{VOCAB_SIZE}")

model.change_vocabulary(new_tokenizer_dir=TOKENIZER_DIR, new_tokenizer_type=TOKENIZER_TYPE)

[NeMo W 2024-03-11 14:12:36 modelPT:258] You tried to register an artifact under config key=tokenizer.model_path but an artifact for it has already been registered.
[NeMo W 2024-03-11 14:12:36 modelPT:258] You tried to register an artifact under config key=tokenizer.vocab_path but an artifact for it has already been registered.
[NeMo W 2024-03-11 14:12:36 modelPT:258] You tried to register an artifact under config key=tokenizer.spe_tokenizer_vocab but an artifact for it has already been registered.


[NeMo I 2024-03-11 14:12:36 mixins:172] Tokenizer SentencePieceTokenizer initialized with 130 tokens
[NeMo I 2024-03-11 14:12:37 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-03-11 14:12:37 rnnt_bpe_models:444] Changed decoder to output to ['<unk>', 'ու', 'ան', 'եր', 'ար', 'ակ', 'ում', '▁է', 'ներ', '▁հ', 'ին', 'այ', 'որ', '▁ե', '▁մ', 'ել', 'ութ', 'ուն', '▁կ', 'ամ', 'ությ', 'ատ', 'ական', '▁ա', '▁ն', 'ած', 'աց', 'աս', 'ով', '▁տ', 'ավ', '▁բ', '▁գ', 'են', 'ություն', 'ա', '▁', 'ն', 'ր', 'ո', 'ե', 'ւ', 'ի', 'մ', 'կ', 'տ', 'յ', 'վ', 'ս', '։', 'լ', 'ց', 'հ', 'ը', 'է', 'թ', 'գ', 'դ', 'ղ', 'ք', 'բ', 'պ', 'ծ', 'շ', ',', 'ռ', 'զ', 'խ', 'չ', 'ջ', 'ձ', 'Ա', 'ժ', 'փ', 'ճ', 'Ն', 'Հ', 'օ', ':', '՝', 'Մ', 'ֆ', 'Ս', 'Բ', 'Կ', '.', 'Ե', '-', 'Գ', 'Դ', 'Պ', 'Տ', 'Ո', 'Ի', '«', '»', 'Վ', 'Շ', '՞', 'Լ', 'Զ', 'Թ', 'Ք', ')', '(', 'Խ', '՛', 'Փ', 'Ծ', 'Ը', 'Ֆ', 'Է', 'Չ', 'Օ', 'Ռ', 'Ձ', 'Յ', 'Ջ', 'Ց', 'Ժ', '՜', 'Ճ', 'Ղ', '՚',

In [39]:
cfg = copy.deepcopy(model.cfg)

# Setup new tokenizer
cfg.tokenizer.dir = TOKENIZER_DIR
cfg.tokenizer.type = "bpe"

# Set tokenizer config
model.cfg.tokenizer = cfg.tokenizer

In [40]:
# Setup train/val/test configs
print(OmegaConf.to_yaml(cfg.train_ds))

manifest_filepath: null
sample_rate: 16000
batch_size: 1
shuffle: true
num_workers: 8
pin_memory: true
use_start_end_token: false
trim_silence: false
max_duration: 20
min_duration: 0.1
is_tarred: false
tarred_audio_filepaths: null
shuffle_n: 2048
bucketing_strategy: fully_randomized
bucketing_batch_size: null



In [41]:
# Setup train, validation, test configs
with open_dict(cfg):
  # Train dataset
  cfg.train_ds.manifest_filepath = f"{train_manifest_full},{dev_manifest}"
  cfg.train_ds.batch_size = 32
  cfg.train_ds.num_workers = 8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.use_start_end_token = False
  cfg.train_ds.trim_silence = True

  # Validation dataset
  cfg.validation_ds.manifest_filepath = test_manifest
  cfg.validation_ds.batch_size = 8
  cfg.validation_ds.num_workers = 8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.use_start_end_token = False
  cfg.validation_ds.trim_silence = True

  # Test dataset
  cfg.test_ds.manifest_filepath = test_manifest
  cfg.test_ds.batch_size = 8
  cfg.test_ds.num_workers = 8
  cfg.test_ds.pin_memory = True
  cfg.test_ds.use_start_end_token = False
  cfg.test_ds.trim_silence = True

In [49]:
# setup model with new configs
model.setup_training_data(cfg.train_ds)
model.setup_multiple_validation_data(cfg.validation_ds)
model.setup_multiple_test_data(cfg.test_ds)

[NeMo I 2024-03-11 14:13:55 collections:196] Dataset loaded with 12702 files totalling 18.82 hours
[NeMo I 2024-03-11 14:13:55 collections:197] 5 files were filtered totalling 0.13 hours


[NeMo W 2024-03-11 14:13:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


[NeMo I 2024-03-11 14:13:55 collections:196] Dataset loaded with 2853 files totalling 4.55 hours
[NeMo I 2024-03-11 14:13:55 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-03-11 14:13:56 collections:196] Dataset loaded with 2853 files totalling 4.55 hours
[NeMo I 2024-03-11 14:13:56 collections:197] 0 files were filtered totalling 0.00 hours


In [50]:
print(OmegaConf.to_yaml(cfg.optim))

name: adamw
lr: 0.0025
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: 15000
  warmup_ratio: null
  min_lr: 1.0e-06



In [51]:
with open_dict(model.cfg.optim):
  model.cfg.optim.lr = 0.025
  model.cfg.optim.weight_decay = 0.001
  model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  model.cfg.optim.sched.warmup_ratio = 0.10  # 10 % warmup
  model.cfg.optim.sched.min_lr = 1e-9

with open_dict(model.cfg.spec_augment):
  model.cfg.spec_augment.freq_masks = 2
  model.cfg.spec_augment.freq_width = 25
  model.cfg.spec_augment.time_masks = 10
  model.cfg.spec_augment.time_width = 0.05

model.spec_augmentation = model.from_config_dict(model.cfg.spec_augment)

In [52]:
use_cer = False
log_prediction = True

model.wer.use_cer = use_cer
model.wer.log_prediction = log_prediction

In [58]:
import torch
import pytorch_lightning as ptl

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'gpu'

EPOCHS = 15  # will take approximately 4 hours

trainer = ptl.Trainer(devices=1,
                      accelerator=accelerator,
                      max_epochs=EPOCHS,
                      accumulate_grad_batches=1,
                      enable_checkpointing=False,
                      logger=False,
                      log_every_n_steps=5,
                      check_val_every_n_epoch=3)

# Setup model with the trainer
model.set_trainer(trainer)

# finally, update the model's internal config
model.cfg = model._cfg

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [57]:
from nemo.utils import exp_manager

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/lang-{LANGUAGE}/',
    name=f"ASR-Model-Language-{LANGUAGE}",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

config = OmegaConf.structured(config)

logdir = exp_manager.exp_manager(trainer, config)
print(logdir)

[NeMo I 2024-03-11 14:14:53 exp_manager:396] Experiments will be logged at experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-03-11_14-14-53
[NeMo I 2024-03-11 14:14:53 exp_manager:856] TensorboardLogger has been set up
experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-03-11_14-14-53


In [59]:
%%time
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-03-11 14:15:03 modelPT:724] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.025
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2024-03-11 14:15:04 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7e80a86356c0>" 
    will be used during training (effective maximum steps = 5955) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    min_lr: 1.0e-09
    max_steps: 5955
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 108 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | joint             | RNNTJoint                         | 822 K 
5 | decoder           | RNNTDecoder                       | 3.4 M 
6 | loss              | RNNTLoss                          | 0     
------------------------------------------------------------------------
4.2 M     Trainable params
108 M     Non-trainable params
112 M     Total params
451.802   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[NeMo W 2024-03-11 14:15:04 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


...

[NeMo I 2024-03-11 15:33:14 wer:334] 
    
[NeMo I 2024-03-11 15:33:14 wer:335] reference:Միջպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամուսնությունները։
[NeMo I 2024-03-11 15:33:14 wer:336] predicted:Միջ պետական հարաբերություններում կարեւոր դերը ընկաղում քաղաքական ամսնությունները։
[NeMo I 2024-03-11 15:33:14 wer:334] 
    
[NeMo I 2024-03-11 15:33:14 wer:335] reference:Այդ օրը այլեւս ոչ մի թռչուն չնկատվեց։
[NeMo I 2024-03-11 15:33:14 wer:336] predicted:Այդ օրը այլեւս ոչ միտրոն չնկաթվեց։
[NeMo I 2024-03-11 15:33:14 wer:334] 
    
[NeMo I 2024-03-11 15:33:14 wer:335] reference:Երբե՜ք այդ մարդը դաժան չէ, այլ...
[NeMo I 2024-03-11 15:33:14 wer:336] predicted:Երբեկ այդ մարդը դաժանչ է, այլ։
[NeMo I 2024-03-11 15:33:15 wer:334] 
    
[NeMo I 2024-03-11 15:33:15 wer:335] reference:Ագրեգատային վիճակը պայմանավորված է մոլեկուլային զանգվածով։
[NeMo I 2024-03-11 15:33:15 wer:336] predicted:Ագրիկատային վիճակը պայմանորված է մոլեկուլային զագվածով։
[NeMo I 2024-03-11 15:33:15 wer:3

Validation: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-03-11 15:53:21 wer:334] 
    
[NeMo I 2024-03-11 15:53:21 wer:335] reference:Միջպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամուսնությունները։
[NeMo I 2024-03-11 15:53:21 wer:336] predicted:Միջ պետական հարաբերություններում կարեւոր դերը ինխաղում քաղաքական ամսնությունները։
[NeMo I 2024-03-11 15:53:21 wer:334] 
    
[NeMo I 2024-03-11 15:53:21 wer:335] reference:Այդ օրը այլեւս ոչ մի թռչուն չնկատվեց։
[NeMo I 2024-03-11 15:53:21 wer:336] predicted:Այդ օրը այլեւս ոչ միտրոն չնկաթվեց։
[NeMo I 2024-03-11 15:53:22 wer:334] 
    
[NeMo I 2024-03-11 15:53:22 wer:335] reference:Երբե՜ք այդ մարդը դաժան չէ, այլ...
[NeMo I 2024-03-11 15:53:22 wer:336] predicted:Երբեկ այդ մարդը դաժանչ է, այլ։
[NeMo I 2024-03-11 15:53:22 wer:334] 
    
[NeMo I 2024-03-11 15:53:22 wer:335] reference:Ագրեգատային վիճակը պայմանավորված է մոլեկուլային զանգվածով։
[NeMo I 2024-03-11 15:53:22 wer:336] predicted:Ագրեկատային վիճակը պայմանորված է մոլեկուլային զանգվածով։
[NeMo I 2024-03-11 15:53:22 wer:

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


CPU times: user 1h 25min 25s, sys: 3min 13s, total: 1h 28min 38s
Wall time: 1h 40min 32s


Please save and download your model.

In [61]:
save_path = f"Model-{LANGUAGE}.nemo"
model.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")

Model saved at path : /content/Model-hy-AM.nemo


In [70]:
trainer.test(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2024-03-11 16:09:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


Testing: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-03-11 16:09:30 wer:334] 
    
[NeMo I 2024-03-11 16:09:30 wer:335] reference:Միջպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամուսնությունները։
[NeMo I 2024-03-11 16:09:30 wer:336] predicted:Միջ պետական հարաբերություններում կարեւոր դերը ինխաղում քաղաքական ամսնությունները։
[NeMo I 2024-03-11 16:09:30 wer:334] 
    
[NeMo I 2024-03-11 16:09:30 wer:335] reference:Այդ օրը այլեւս ոչ մի թռչուն չնկատվեց։
[NeMo I 2024-03-11 16:09:30 wer:336] predicted:Այդ օրը այլեւս ոչ միտրոն չնկաթվեց։
[NeMo I 2024-03-11 16:09:31 wer:334] 
    
[NeMo I 2024-03-11 16:09:31 wer:335] reference:Երբե՜ք այդ մարդը դաժան չէ, այլ...
[NeMo I 2024-03-11 16:09:31 wer:336] predicted:Երբեկ այդ մարդը դաժանչ է, այլ։
[NeMo I 2024-03-11 16:09:31 wer:334] 
    
[NeMo I 2024-03-11 16:09:31 wer:335] reference:Ագրեգատային վիճակը պայմանավորված է մոլեկուլային զանգվածով։
[NeMo I 2024-03-11 16:09:31 wer:336] predicted:Ագրեկատային վիճակը պայմանորված է մոլեկուլային զանգվածով։
[NeMo I 2024-03-11 16:09:31 wer:

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        global_step        │     5955.00048828125      │
│         test_wer          │    0.5285160541534424     │
└───────────────────────────┴───────────────────────────┘

[{'global_step': 5955.00048828125, 'test_wer': 0.5285160541534424}]

[{'global_step': 5955.00048828125, 'test_wer': 0.5285160541534424}]

model - nvidia/stt_en_fastconformer_transducer_large

remove punctuation and make all text lowercase

epochs - 15
